# Ethereum and ERC20 derivation and transaction manual

In [1]:
from web3 import Web3, middleware
from web3.gas_strategies.time_based import fast_gas_price_strategy

from crypto_eth import HDPrivateKey, HDKey
import binascii

## Direvation

In [3]:
master_key = HDPrivateKey.master_key_from_mnemonic(
    'trade icon company use feature fee order double inhale gift news long')
root_keys = HDKey.from_path(master_key, "m/44'/60'/0'")
acct_priv_key = root_keys[-1]
private_keys = []

# derivation example for Ethereum generate first 10 addresses
for i in range(10):
    keys = HDKey.from_path(acct_priv_key, '{change}/{index}'.format(change=0, index=i))
    private_key = keys[-1]
    public_key = private_key.public_key
    private_keys.append(private_key)
    print(f'Index {i}:')
    print(f'\tPrivate key (hex, compressed): {private_key._key.to_hex()}')
    print(f'\tAddress: {private_key.public_key.address()}')

Index 0:
	Private key (hex, compressed): 396f7f85659d2b43ef05f7abc80586aa3989e8ef7218433350ddd5945f7f9008
	Address: 0x88a007ec4f1819f24c0988fc9c26496b99b436d1
Index 1:
	Private key (hex, compressed): 7f31b8e44f7cdf90959ac38f412698640ed4113414e4c0bf074cb3aee9cec72a
	Address: 0xab33d517b6a63a0b1c099b8438d6641cf1a984cc
Index 2:
	Private key (hex, compressed): 49edccc09e07e63daf689fd808886837f18cfb87900f5957afea0f633d73c2f8
	Address: 0xb76d3a5d568f339411a6f0697694cddb6396df58
Index 3:
	Private key (hex, compressed): 015e90cb0d2f1bebfee8248c1991e13478aaf68279bbd8d7eee839c55f8499b5
	Address: 0xcb64cd644d24b57a96287f61dffeb402b01e23c6
Index 4:
	Private key (hex, compressed): 4752f5feb87ff6431567443ef4243f2d011721f0ed76ce0ef274ca3180aabbfd
	Address: 0x8396738b4cb2a2a7ca636161346a7cb22ee06e25
Index 5:
	Private key (hex, compressed): 857fdcbcfe1bd3620a699b6251bd37e9b0bb9b0ccb3fd570b0beb9a90133344f
	Address: 0xbd666c0ec86628475b1602aba0ebf45570a33d4d
Index 6:
	Private key (hex, compressed): 025fe

## Ethereum transction
### Setup Web3

In [5]:
# use infura endpoint
# endpoint_url = 'wss://ropsten.infura.io/ws/adc06f70568e46d88376a8a3a30e5497'
endpoint_url = 'https://ropsten.infura.io/v3/adc06f70568e46d88376a8a3a30e5497'

# Ropsten chainId is 3
chainId = 3

# setup connection
w3 = Web3(Web3.HTTPProvider(endpoint_url))
if not w3.isConnected():
    print('Can\'t connect to Web3 endpoint')

# select gas price strategy
w3.eth.setGasPriceStrategy(fast_gas_price_strategy)

w3.middleware_onion.add(middleware.time_based_cache_middleware)
w3.middleware_onion.add(middleware.latest_block_based_cache_middleware)
w3.middleware_onion.add(middleware.simple_cache_middleware)

### Get last block

In [6]:
block = w3.eth.getBlock('latest')
print(f'Current block: {block}')

Current block: AttributeDict({'difficulty': 3914584447, 'extraData': HexBytes('0xd983010906846765746889676f312e31312e3133856c696e7578'), 'gasLimit': 8000000, 'gasUsed': 208966, 'hash': HexBytes('0x590b7020a48057ed8b2ce95c4a112d1f6d211776bc09a2aed2ac18d6b8d5a75b'), 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000040000000000000000000000800000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000800000020000000000000000000000000000000000'), 'miner': '0x4CCfB3039b78d3938588157564c9AD559bAfAB94', 'mixHash': HexBytes('0xc1dc846fc6de9e1f32562b01685b746a93cf0ff5c713c9cd2cad95df3d85fd85'), 'nonce': HexBytes('0x491f293803d7d391'), 'number

### Prepare accounts

In [7]:
# take first two accounts as from and to
from_address = w3.toChecksumAddress(private_keys[0].public_key.address())
from_private_key = private_keys[0]._key.to_hex()

to_address = w3.toChecksumAddress(private_keys[1].public_key.address())
to_private_key = private_keys[1]._key.to_hex()

### Send Ethereum

In [11]:
# fetching nonce (count of transactions)
nonce = w3.eth.getTransactionCount(from_address)

transaction = {
    'to': to_address,
    'value': 200_000, # 0.0000000000002 Ether
    'gas': 21_000,
    'gasPrice': w3.eth.generateGasPrice(),
    'nonce': nonce,
    'chainId': chainId
}

signed = w3.eth.account.sign_transaction(transaction, from_private_key)

print(f'Signed transaction: {binascii.b2a_hex(signed.rawTransaction)}')

transaction_hash = w3.eth.sendRawTransaction(signed.rawTransaction)

print(f'Transaction hash: {transaction_hash}')

Signed transaction: b'f8688203f4844aa797c382520894ab33d517b6a63a0b1c099b8438d6641cf1a984cc83030d408029a031396762305677c6cdf324d3c50d3b939d7c4102930ff97d7d696ae188ebc317a036a8d66c4be95dddb14d41b90d8bbc25017acb927da5df3764085e91bde01b65'
Transaction hash: b"q\xc2.\xc2\xb0\xbc\x83\xdf\x18\xd1\x9a\x1a\x98\xd0Y \x13'QY\xdbY\xe0\xdb\x7f{\x97\x1c\xb4\xf0u\xe1"


### Ethereum balance

In [16]:
print(f" from_address balance: {w3.fromWei(w3.eth.getBalance(from_address), 'ether')}")
print(f" to_address balance: {w3.fromWei(w3.eth.getBalance(to_address), 'ether')}")

 from_address balance: 1.008429215369994551
 to_address balance: 1.002326564527503776


### Send ERC20

In [8]:
# read abi for ERC20 contract
with open('erc20_abi.json', 'r') as file:
    erc20_abi = file.read()
    
print(erc20_abi)

[
    {
        "constant": true,
        "inputs": [],
        "name": "name",
        "outputs": [
            {
                "name": "",
                "type": "string"
            }
        ],
        "payable": false,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": false,
        "inputs": [
            {
                "name": "_spender",
                "type": "address"
            },
            {
                "name": "_value",
                "type": "uint256"
            }
        ],
        "name": "approve",
        "outputs": [
            {
                "name": "",
                "type": "bool"
            }
        ],
        "payable": false,
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "constant": true,
        "inputs": [],
        "name": "totalSupply",
        "outputs": [
            {
                "name": "",
                "type": "uint256"
            }
  

In [10]:
# Another Test Coin (ATC) ERC20
erc20_contract_address = "0x96f55B7d683EB46FF23aBEf2aFb41EB54FCF524b"
erc20_contract = w3.eth.contract(address=erc20_contract_address, abi=erc20_abi)

# fetching nonce (count of transactions)
nonce = w3.eth.getTransactionCount(from_address)

erc20_transaction = erc20_contract.functions.transfer(erc20_contract_address, 1).buildTransaction(
    {
        'chainId': chainId,
        'gas': 100_000,
        'gasPrice': w3.eth.generateGasPrice(),
        'nonce': nonce,
    })

w3.eth.generateGasPrice()

print(erc20_transaction)

{'value': 0, 'chainId': 3, 'gas': 100000, 'gasPrice': 1238787536, 'nonce': 1012, 'to': '0x96f55B7d683EB46FF23aBEf2aFb41EB54FCF524b', 'data': '0xa9059cbb00000000000000000000000096f55b7d683eb46ff23abef2afb41eb54fcf524b0000000000000000000000000000000000000000000000000000000000000001'}


### ERC20 balance

In [21]:
decimals = 10 ** erc20_contract.functions.decimals().call()
from_address_erc20_balance = erc20_contract.functions.balanceOf(from_address).call()
to_address_erc20_balance = erc20_contract.functions.balanceOf(to_address).call()


print(f" from_address balance: {from_address_erc20_balance / decimals}")
print(f" to_address balance: {to_address_erc20_balance / decimals}")

 from_address balance: 6689.27329999
 to_address balance: 179.31569617
